In [ ]:
import pickle

In [ ]:
f1 = open('true_face_haar.pickle', 'rb')
f2 = open('inferred_face_haar.pickle', 'rb')
f3 = open('better_face_extraction.pickle', 'rb')

In [ ]:
true_dict = pickle.load(f1)
inferred_dict = pickle.load(f2)
better_dict = pickle.load(f3)

## FPS

In [ ]:
true_list = []
for key, value in true_dict.items():
    true_list.append(value['sec'])
true_list = [1/item for item in true_list[1:]] # discard 1st frame for cold start latency

In [ ]:
inferred_list = []
for key, value in inferred_dict.items():
    inferred_list.append(value['sec'])
inferred_list = [1/item for item in inferred_list[1:]] # discard 1st frame for cold start latency

In [ ]:
better_list = []
for key, value in better_dict.items():
    better_list.append(value['sec'])
better_list = [1/item for item in better_list[1:]] # discard 1st frame for cold start latency

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame([true_list, inferred_list, better_list]).transpose()
df.columns = ['true','inferred','better']
df.head()

In [ ]:
# df.describe().to_html()

In [ ]:
data = [{
    'x': df.index,
    'y': df[col],
    'name': col
}  for col in df.columns]

In [ ]:
layout = go.Layout(
    title='Face Extraction (true v. inferred v. better)',
    yaxis=dict(       
        title='fps',
    ),
    xaxis=dict(
        title='frame number',
    )
)

In [ ]:
fig = go.Figure(data=data, layout=layout)

In [ ]:
# py.iplot(fig)

## Accuracy

In [ ]:
appearances = {}
f = open('clip-Appearances.txt','r')
for x in f:
    split = x.strip('\n').split(',')
    actor = int(split[0])
    appearances[actor] = [int(split[1]), int(split[2])]

In [ ]:
appearances

In [ ]:
# 'better' found

tp = 0 # face there, face detected
fn = 0 # face there, face not detected

for actor in appearances:
    for fnum in range(appearances[actor][0], appearances[actor][1]):
        
        face_found = better_dict[fnum]['coords_list']
        
        if len(face_found)>0: # detected
            tp += 1
        else: # not detected
            fn += 1

In [ ]:
(tp, fn)

## Accuracy (Part 2)

In [ ]:
yes_face = []
for actor in appearances:
    for fnum in range(appearances[actor][0], appearances[actor][1]):
        yes_face.append(fnum)

In [ ]:
all_frames = list(better_dict.keys())
len(all_frames)

In [ ]:
no_face = list(set(all_frames) - set(yes_face))
len(yes_face), len(no_face)

In [ ]:
tn = 0 # face not there, face not detected
fp = 0 # face not there, face detected

for fnum in better_dict.keys():
    if fnum in no_face:
        
#         face_found = true_dict[fnum]['coords_list']
#         face_found = inferred_dict[fnum]['coords_list']
        face_found = better_dict[fnum]['coords_list']

        if len(face_found)>0: # detected
            fp += 1
        else: # not detected
            tn += 1

In [ ]:
(tn, fp)

In [ ]:
# (687, 7) for true
# (694, 0) for inferred
# (694, 0) for better